In [1]:
import numpy as np

def generate_rotating_wedge_stimulus(n_frames=300,
                                     height=200,
                                     width=200,
                                     visual_field_deg=16.08,
                                     wedge_width_deg=90,
                                     cycle_duration=32,
                                     n_cycles=8,
                                     blank_duration=22,
                                     direction='ccw'):
    """
    Generates a binary stimulus matrix representing a rotating wedge stimulus.

    Parameters:
    - n_frames: total number of frames (TRs), default 300
    - height, width: spatial resolution in pixels
    - visual_field_deg: total size of visual field in degrees (assumed square)
    - wedge_width_deg: angular width of the rotating wedge
    - cycle_duration: seconds per full 360° rotation (1 cycle)
    - n_cycles: number of full cycles
    - blank_duration: seconds of blank at start and end
    - direction: 'ccw' or 'cw' (counterclockwise or clockwise)

    Returns:
    - stimulus: array of shape (n_frames, height, width) with binary apertures
    """
    # Visual field coordinate grid in degrees
    x = np.linspace(-visual_field_deg / 2, visual_field_deg / 2, width)
    y = np.linspace(-visual_field_deg / 2, visual_field_deg / 2, height)
    xv, yv = np.meshgrid(x, y)
    polar_angle = (np.arctan2(yv, xv) * 180 / np.pi) % 360  # 0 to 360

    # TRs
    tr = 1.0
    frames_per_cycle = int(cycle_duration / tr)
    blank_frames = int(blank_duration / tr)
    stimulus = np.zeros((n_frames, height, width))

    # Fill in the rotating wedge frames
    for t in range(blank_frames, n_frames - blank_frames):
        cycle_pos = (t - blank_frames) % frames_per_cycle
        angle = (360 * cycle_pos / frames_per_cycle) % 360
        if direction == 'cw':
            angle = (360 - angle) % 360  # reverse rotation

        angle_diff = (polar_angle - angle + 180) % 360 - 180
        wedge_mask = (np.abs(angle_diff) <= wedge_width_deg / 2)
        stimulus[t][wedge_mask] = 1.0

    return stimulus

# Example usage
stimulus_matrix = generate_rotating_wedge_stimulus()
print(stimulus_matrix.shape)  

(300, 200, 200)


In [1]:
def make_prf_matrix(x0_pix, y0_pix, sigma_pix, height=200, width=200):
    """
    Generate a 2D Gaussian pRF matrix over a 200x200 grid (pixel units).
    
    Parameters:
    - x0_pix, y0_pix: center of the pRF in pixels (0-199)
    - sigma_pix: standard deviation of the Gaussian in pixels
    - height, width: grid size (default 200x200)
    
    Returns:
    - prf: 2D array of shape (height, width)
    """
    x = np.linspace(0, width - 1, width)
    y = np.linspace(0, height - 1, height)
    xv, yv = np.meshgrid(x, y)

    prf = np.exp(-((xv - x0_pix)**2 + (yv - y0_pix)**2) / (2 * sigma_pix**2))
    return prf

In [3]:
%pip install nilearn
import nibabel as nib
fmri_file = "./hcp_retino/HCP7T_retino/tfMRI_RETBAR1_7T_AP.dtseries.nii"

# Load the file
img = nib.load(fmri_file)
data = img.get_fdata()

print("Shape of fMRI data:", data.shape)

Note: you may need to restart the kernel to use updated packages.
Shape of fMRI data: (300, 91282)


In [4]:
def simulate_voxel_response(stimulus, prf):
    """
    Simulate a voxel's neural response over time by multiplying the stimulus with a pRF.

    Parameters:
    - stimulus: 3D array of shape (T, H, W), where T = timepoints (TRs)
    - prf: 2D array of shape (H, W), the pRF matrix

    Returns:
    - response: 1D array of shape (T,), predicted neural response over time
    """
    # Dot product over (H, W) for each timepoint
    response = np.tensordot(stimulus, prf, axes=([1, 2], [0, 1]))
    return response